In [1]:
# fix seeds 
SEED = 42

import random as rn
rn.seed(SEED)

import numpy as np
np.random.seed(SEED)

import tensorflow as tf
tf.random.set_seed(SEED)

import sys
sys.path.append("..")

In [5]:
from src.model.resnet50 import ResNet50
from src.dataset.segmentation_dataset import SegmentationDataset

dataset = SegmentationDataset(
    '/workspace/data/prepped-binary/segmentation-train-10.tfrecord',
    nb_class=1,
    one_pixel_mask=True,
    #one_image_label=True,
    shuffle_buffer_size=0
).dataset

for b in dataset:
    img, labels, mask = b
    print(tf.where(labels==1))
    print(tf.where(mask))
# model = ResNet50(
#     input_shape = (128, 128, 12), 
#     classes = 1
# )
# print(model.summary())
# model.compile(
#     optimizer='sgd',
#     loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#     metrics='binary_accuracy'
# )
# model.fit(
#     dataset,
#     epochs=10,
# )
                   
# model.predict(dataset)

tf.Tensor(
[[  5  22 115   0]
 [  6  37  88   0]], shape=(2, 4), dtype=int64)
tf.Tensor(
[[  0  45  48]
 [  1 118  65]
 [  2  68 124]
 [  3  68 104]
 [  4  10  84]
 [  5  22 115]
 [  6  37  88]
 [  7  71  89]
 [  8  89  13]
 [  9  59  66]], shape=(10, 3), dtype=int64)


In [6]:
import sys
sys.path.append("..")

from src.model.unet import Unet
from src.dataset.segmentation_dataset import SegmentationDataset
import tensorflow as tf

dataset = SegmentationDataset(
    '/workspace/data/prepped-binary/segmentation-train-10.tfrecord',
    nb_class=1,
    #one_image_label=True,
    shuffle_buffer_size=0
).dataset

model = Unet(
    input_shape=(128, 128, 12),
    classes=1,  
    encoder_weights=None,
    encoder_freeze=True,
    one_image_label=True,
    activation='sigmoid'
)
# model.compile(
#     optimizer='sgd',
#     loss=tf.keras.losses.BinaryCrossentropy(),#from_logits=True),
#     metrics='binary_accuracy',
#     weighted_metrics=['binary_accuracy']
# )
# model.fit(
#     dataset,
#     epochs=5,
# )
# model.predict(dataset)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 12 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 12) 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 64, 64)   37632       conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 64, 64, 64)   256         conv1[0][0]                      
____________________________________________________________________________________________